In [106]:
import process_output
from PIL import Image, ImageEnhance, ImageFilter
import requests
from io import BytesIO
import imgkit
import json

In [107]:
def get_unsplash_url(client_id, query, orientation):
    
    root = 'https://api.unsplash.com/'
    path = 'photos/random/?client_id={}&query={}&orientation={}'
    search_url = root + path.format(client_id, query, orientation)
    
    api_response = requests.get(search_url)
    data = api_response.json()
    api_response.close()
    #print(json.dumps(data, indent=4, sort_keys=True))
    
    return data['urls']['regular']

client_id = 'L-CxZwGQjlKToJ1xdSiBCnj1gAyUJ0nBLKYqaQOXOAg'
query = 'nature dark'
orientation = 'landscape'

image_url = get_unsplash_url(client_id, query, orientation)

In [108]:
quote_text = process_output.get_quote()

image_response = requests.get(image_url)
img = Image.open(BytesIO(image_response.content))
image_response.close()

# resize down until either a desired width or height is acheived, then crop the other dimension
# to acheive a non-distorted version of the image with desired dimensions
def resize_crop(im, desired_width=800, desired_height=600):
    
    width, height = im.size
    if width/height > desired_width/desired_height:
        im.thumbnail((width, desired_height))
    else:
        im.thumbnail((desired_width, height))
        
    width, height = im.size
    box = [0, 0, width, height] # left, upper, right, lower
    if width > desired_width:
        box[0] = width/2 - desired_width/2
        box[2] = width/2 + desired_width/2
    if height > desired_height:
        box[1] = height/2 - desired_height/2
        box[3] = height/2 + desired_height/2
    
    im = im.crop(box=box)
    return im

def reduce_color(im, desired_color=0.5):
    converter = ImageEnhance.Color(im)
    im = converter.enhance(desired_color)
    return im

def gaussian_blur(im, radius=2):
    im = im.filter(ImageFilter.GaussianBlur(radius=radius))
    return im

img = resize_crop(img)
img = reduce_color(img)
#img = gaussian_blur(img)
img.save('templates-and-assets/backdrop.jpg')

In [109]:
html_doc = None

with open('templates-and-assets/image_template.html', 'r') as f:
    html_doc = f.read()

html_doc = html_doc.replace('dynamictext', quote_text)

#print(len(quote_text))

def get_font_size(text):
    size = len(text)
    if size < 40:
        return '44'
    if size < 75:
        return '36'
    return '30'

html_doc = html_doc.replace('dynamicfontsize', get_font_size(quote_text))

with open('templates-and-assets/image_out.html', 'w') as f:
    f.write(html_doc)

In [110]:
imgkit.from_file('templates-and-assets/image_out.html',
                 'templates-and-assets/image_out.jpg',
                 options={'width' : 800, 'height' : 600,
                          'quality' : 100, 'encoding' : 'utf-8'})

Loading page (1/2)
Rendering (2/2)                                                    
Done                                                               


True